In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
os.environ['THEANO_FLAGS'] = "device=cpu"

In [ ]:
import copy
import shelve
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.gridspec import GridSpec
from IPython.display import display

plt.rcParams['savefig.dpi'] = 300

In [ ]:
from experiments.hmmvsrnn_reco.a_data import tmpdir, frame_seqs, dataset, vocabulary, get_ref_pts, detect_invalid_pts, interpolate_positions
from sltools.transform import Transformation, transform_pose2d, transform_pose3d, transform_frames

tmpdir = tmpdir.split('.')[0]
tgt_dist = 2
joints = dataset.JointType

flip_mapping = ([joints.ShoulderRight, joints.ElbowRight,
                 joints.WristRight, joints.HandRight, joints.ShoulderLeft,
                 joints.ElbowLeft, joints.WristLeft, joints.HandLeft,
                 joints.HipRight, joints.KneeRight, joints.AnkleRight,
                 joints.FootRight, joints.HipLeft, joints.KneeLeft,
                 joints.AnkleLeft, joints.FootLeft],
                [joints.ShoulderLeft, joints.ElbowLeft,
                 joints.WristLeft, joints.HandLeft, joints.ShoulderRight,
                 joints.ElbowRight, joints.WristRight, joints.HandRight,
                 joints.HipLeft, joints.KneeLeft, joints.AnkleLeft,
                 joints.FootLeft, joints.HipRight, joints.KneeRight,
                 joints.AnkleRight, joints.FootRight])

video = dataset.bgr_frames(0)
poses_2d = dataset.positions(0)
poses_3d = dataset.positions_3d(0)
invalid_masks = detect_invalid_pts(poses_2d)
poses_2d = interpolate_positions(poses_2d, invalid_masks)
poses_3d = interpolate_positions(poses_3d, invalid_masks)
ref2d = get_ref_pts(poses_2d)
ref3d = get_ref_pts(poses_3d)

zshifts = np.mean(tgt_dist - ref3d[:, 2])

In [ ]:
# transformation = Transformation(
#     ref2d=ref2d, ref3d=ref3d, flip_mapping=flip_mapping,
#     frame_width=640,
#     fliplr=False,
#     tilt=5 * np.pi / 180,
#     zshift=zshifts,
#     xscale=1.15, yscale=0.85,
#     zscale=1, tscale=1)

# t = 10
# plt.figure()
# plt.imshow(video[t])
# plt.scatter(poses_2d[t, :, 0], poses_2d[t, :, 1])
# plt.figure()
# plt.imshow(transform_frames(video, transformation)[t])
# trans_pose2d = transform_pose2d(poses_2d, transformation)
# plt.scatter(trans_pose2d[t, :, 0], trans_pose2d[t, :, 1])
# plt.show()

In [ ]:
[tmpdir, tmpdir + '.run1', tmpdir + '.run2']

In [ ]:
[(e[1], e[4]['ji_filtered']) for e in experiments if e[0] == "hmm_skel_tc21"]

In [ ]:
experiments = []

for run, directory in enumerate([tmpdir + '.run1', tmpdir + '.run2', tmpdir + '.run3']):
    for report_file in os.listdir(directory):
        if not report_file.endswith(".dat"):
            continue
        
        f = os.path.join(directory, report_file[:-4])
        with shelve.open(f, flag='r') as report:
            if 'analysis' not in report.keys():
                continue
            meta = report['meta']
            name = meta['experiment_name']
            args = report['args']['encoder_kwargs']
            analysis = report['analysis']
            experiments.append((name, run, meta, args, analysis))

In [ ]:
sorted([(e[0], e[4]['ji_filtered'][2]) for e in experiments])

# Varying TC size

In [ ]:
dtype = [('model', 'S3'), 
         ('win', 'i'),
         ('nparms', 'i'),
         ('ji', 'f'),
         ('acc', 'f')]

tc_analyses = np.sort([
    np.array((
        m['model'],
        a['filter_dilation'] * (a['tconv_sz'] - 1) + 1,
        a['tconv_sz'] * a['num_tc_filters'],
        r['ji_filtered'][1], 
        r['accuracy_filtered'][1]), 
        dtype=dtype)
    for _, _, m, a, r in experiments 
    if m['modality'] == "skel"])

plt.figure(dpi=100) 

legend = []
subset = (tc_analyses['model'] == b"rnn")
p1 = plt.scatter(
    tc_analyses[subset]['win'],
    tc_analyses[subset]['ji'],
    s=tc_analyses[subset]['nparms'] / 50,
    marker="o", alpha=0.5)
    
legend = []
subset = (tc_analyses['model'] == b"hmm")
p2 = plt.scatter(
    tc_analyses[subset]['win'],
    tc_analyses[subset]['ji'],
    s=tc_analyses[subset]['nparms'] / 50,
    marker="o", alpha=0.5)

plt.legend([p1, p2], ['rnn', 'hmm'], loc='best')

plt.xlabel("window size")
plt.ylabel("Jaccard Index")
plt.xticks(np.arange(3, 32, 4))
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.show()

# Transfer learning

In [ ]:
source_experiments = {
    (name, run): (meta['modality'], analysis['accuracy_filtered'][2], analysis['ji_filtered'][2])
    for name, run, meta, args, analysis in experiments 
    if meta['modality'] != "transfer"}

columns = [
    'model', 'modality', 'terminate_at', 'run', 
    'acc', 'delta_ref_acc', 'delta_other_acc', 'ji', 'delta_ref_ji', 'delta_other_ji']

transfer_analyses = []

for name, run, meta, args, analysis in experiments:
    if meta['modality'] != 'transfer':
        continue

    model = meta['model']
    terminate_at = args['terminate_at']
    modality, acc_other, ji_other = source_experiments[(args['transfer_from'], run)]
    _, acc_ref, ji_ref = source_experiments[(model + args['transfer_from'][3:], run)]
    acc = analysis['accuracy_filtered'][2]
    ji = analysis['ji_filtered'][2]
    transfer_analyses.append(
        (model, modality, terminate_at, run, 
                  acc, acc - acc_ref, acc - acc_other, 
                  ji, ji - ji_ref, ji - ji_other))

transfer_analyses = pd.DataFrame(transfer_analyses, columns=columns)

In [ ]:
transfer_analyses.groupby(['model', 'modality', 'terminate_at']).mean()

In [ ]:
# compare confusion on video frames

hmm_conf = np.zeros((21, 21))
rnn_conf = np.zeros((21, 21))

for rundir in [tmpdir + '.run1', tmpdir + '.run2', tmpdir + '.run3']:
    with shelve.open(os.path.join(rundir, "hmm_bgr_tc15"), flag='r') as report:
        hmm_analysis = report['analysis']
        hmm_conf += hmm_analysis['confusion_filtered'][1]

    with shelve.open(os.path.join(rundir, "rnn_bgr_tc15"), flag='r') as report:
        rnn_analysis = report['analysis']
        rnn_conf += rnn_analysis['confusion_filtered'][1]

hmm_conf /= np.sum(hmm_conf, axis=1, keepdims=True)
rnn_conf /= np.sum(rnn_conf, axis=1, keepdims=True)

conf_diff = hmm_conf - rnn_conf

# plot
plt.figure(dpi=150, figsize=(6, 3))
limits = np.max(abs(conf_diff))
plt.imshow(
    conf_diff, 
    clim=(-limits, limits), 
    cmap='RdBu')
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'],
    fontsize=8)
plt.xticks(np.arange(0, 21), [''] * 21)
plt.gca().annotate(
    '', 
    xy=(1.5, 0.1), xycoords='axes fraction', xytext=(1.5, 0.9), 
    arrowprops=dict(arrowstyle="<->", color='k'))
plt.gca().annotate(
    'rnn', xy=(1.47, 0.05), xycoords='axes fraction', xytext=(1.47, 0.05))
plt.gca().annotate(
    'hmm', xy=(1.44, 0.92), xycoords='axes fraction', xytext=(1.44, 0.92))
plt.colorbar()

# Compare misclassification
a = np.sum(hmm_conf[1:, 1:]) - np.sum(np.diag(hmm_conf[1:, 1:]))
b = np.sum(rnn_conf[1:, 1:]) - np.sum(np.diag(rnn_conf[1:, 1:]))
print(a, b, (a - b) / b)

In [ ]:
# compare confusion on body poses

hmm_conf = np.zeros((21, 21))
rnn_conf = np.zeros((21, 21))

for rundir in [tmpdir + '.run1', tmpdir + '.run2', tmpdir + '.run3']:
    with shelve.open(os.path.join(rundir, "hmm_skel_tc15"), flag='r') as report:
        hmm_analysis = report['analysis']
        hmm_conf += hmm_analysis['confusion_filtered'][1]

    with shelve.open(os.path.join(rundir, "rnn_skel_tc15"), flag='r') as report:
        rnn_analysis = report['analysis']
        rnn_conf += rnn_analysis['confusion_filtered'][1]

hmm_conf /= np.sum(hmm_conf, axis=1, keepdims=True)
rnn_conf /= np.sum(rnn_conf, axis=1, keepdims=True)

conf_diff = hmm_conf - rnn_conf

# plot
plt.figure(dpi=150, figsize=(6, 3))
limits = np.max(abs(conf_diff))
plt.imshow(
    conf_diff, 
    clim=(-limits, limits), 
    cmap='RdBu')
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'],
    fontsize=8)
plt.xticks(np.arange(0, 21), [''] * 21)
plt.gca().annotate(
    '', 
    xy=(1.5, 0.1), xycoords='axes fraction', xytext=(1.5, 0.9), 
    arrowprops=dict(arrowstyle="<->", color='k'))
plt.gca().annotate(
    'rnn', xy=(1.47, 0.05), xycoords='axes fraction', xytext=(1.47, 0.05))
plt.gca().annotate(
    'hmm', xy=(1.44, 0.92), xycoords='axes fraction', xytext=(1.44, 0.92))
plt.colorbar()

# Compare misclassification
a = np.sum(hmm_conf[1:, 1:]) - np.sum(np.diag(hmm_conf[1:, 1:]))
b = np.sum(rnn_conf[1:, 1:]) - np.sum(np.diag(rnn_conf[1:, 1:]))
print(a, b, (a - b) / b)

# plt.tight_layout(rect=[0, 0., 1.2, 1])
# plt.savefig("/home/granger/exp1_confdiff_skel.pdf", bbox_inches='tight')

In [ ]:
# visualize mistaken classes

plt.figure(dpi=150)

plt.imshow(
    np.clip(rnn_conf, 0.0001, 1), 
    clim=(0.001, 1),
    norm=colors.LogNorm(vmin=0.0001, vmax=1., clip=True))
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'])
plt.xticks(np.arange(0, 21), [''] * 21)
plt.colorbar()

plt.tight_layout()
plt.savefig("/home/granger/exp1_rnn_pose_confusion.pdf", bbox_inches='tight')

In [ ]:
# np.random.seed(42)

c1 = 16
c2 = 0

fig = plt.figure(figsize=(8, 5), dpi=150)
all_glosses = np.array([[seq] + list(g) for seq in dataset.default_splits()[0] for g in dataset.glosses(seq)])


p = (all_glosses[:, 1] == c1) / np.sum(all_glosses[:, 1] == c1)
seq1, c1, start1, stop1 = all_glosses[np.random.choice(len(all_glosses), p=p)]
vid1 = dataset.bgr_frames(seq1)

for i, t in enumerate(np.linspace(start1 + 10, stop1 - 10, 5).astype(np.int)):
    frame = vid1[t]
    pose = dataset.positions(seq1)[t]
    x1, x2, y1, y2 = np.min(pose[:, 0]) - 30, np.max(pose[:, 0]) + 30, np.min(pose[:, 1]) - 20, np.max(pose[:, 1]) - 130
    ax = fig.add_subplot(2, 5, i + 1)
    ax.imshow(frame[y1:y2, x1:x2])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

p = (all_glosses[:, 1] == c2) / np.sum(all_glosses[:, 1] == c2)
seq2, c2, start2, stop2 = all_glosses[np.random.choice(len(all_glosses), p=p)]
vid2 = dataset.bgr_frames(seq2)

for i, t in enumerate(np.linspace(start2 + 10, stop2 - 10, 5).astype(np.int)):
    frame = vid2[t]
    pose = dataset.positions(seq2)[t]
    x1, x2, y1, y2 = np.min(pose[:, 0]) - 30, np.max(pose[:, 0]) + 30, np.min(pose[:, 1]) - 20, np.max(pose[:, 1]) - 130
    ax = fig.add_subplot(2, 5, i + 6)
    ax.imshow(frame[y1:y2, x1:x2])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.tight_layout(pad=0, h_pad=0)

In [ ]:
# file = "/home/granger/dev/code_these/datasets/ch14dataset/data/Sample0001_color.mp4"
# 
# class Video(Sequence):
#     def __init__(self, file):
#         self.file = file
#         self.container = av.open(file)
#         self.stream = self.container.streams.get(video=0)[0]
#         self.frame_base = self.stream.time_base * self.stream.average_rate
#         self.packet_iter = self.container.demux(self.stream)
#         self.last_packet = next(self.packet_iter).decode()
# 
#         self.offset = 0
#         self.duration = int(self.stream.duration * self.stream.time_base
#                             * self.stream.average_rate)
# 
#     def __len__(self):
#         return self.duration
# 
#     def __getitem__(self, t):
#         # slicing support
#         if isinstance(t, slice):
#             start, stop, step = t.start, t.stop, t.step
# 
#             # defaults
#             start = start or 0
#             stop = stop or -1
#             step = step or 1
# 
#             # range check
#             if step != 1:
#                 raise IndexError("Video slicing is limited to step 1")
#             if start < -len(self) or start >= len(self) \
#                     or stop < -len(self) - 1 or stop > len(self):
#                 raise IndexError("Video slice index out of range.")
# 
#             # negative indexing
#             start = start + len(self) if start < 0 else start
#             stop = stop + len(self) if stop < 0 else stop
#             stop = max(start, stop)
# 
#             video = Video(self.file)
#             video.offset = self.offset + start  # cumulate offsets
#             video.duration = stop - start
# 
#             return video
# 
#         # range check
#         if t < -len(self) or t >= len(self):
#             raise IndexError("Video index out of range.")
# 
#         # negative indexing
#         if t < 0:
#             t += len(self)
# 
#         t += self.offset
# 
#         t_pts = t / self.stream.time_base / self.stream.average_rate
# 
#         # Do seeking if needed
#         if t > self.last_packet[-1].pts * self.frame_base + len(self.last_packet) \
#                 or t < self.last_packet[0].pts * self.frame_base:
#             self.stream.seek(int(t / self.frame_base))
#             self.packet_iter = self.container.demux(self.stream)
# 
#         while True:
#             for f in self.last_packet:
#                 if f.pts == t_pts:
#                     return f.to_rgb().to_nd_array()
#             self.last_packet = next(self.packet_iter).decode()
# 
# v = Video(file)
# 
# assert np.all(np.stack(list(v[10:20])) == np.stack(list(v)[10:20]))
# assert np.all(np.stack([v[i] for i in range(0, 40, 3)]) == np.stack(list(v)[0:40:3]))
# 
# plt.imshow(v[100])
# plt.show()